#### Downloading & Inspecting EUMETSAT/MSG Data

Code adapted from https://eumetsatspace.atlassian.net/wiki/spaces/EUMDAC/pages/1760198661/Python+Library

Requires signing up for an account with EUMETSAT EO Portal: https://eoportal.eumetsat.int/userMgmt/
Once done, the consumer key and secret can be found under the user information: https://eumetsatspace.atlassian.net/wiki/spaces/DSEV/pages/375652353/API+Authentication

In [1]:
import autoroot

In [79]:
import os
import ast
import eumdac
import datetime
import shutil
import pyproj
import rioxarray
import rasterio
import xarray as xr

from pyproj import CRS

In [3]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Insert your personal key and secret into the single quotes
consumer_key = ""
consumer_secret = ""

credentials = (consumer_key, consumer_secret)

token = eumdac.AccessToken(credentials)

print(f"This token '{token}' expires {token.expiration}")

In [ ]:
datastore = eumdac.DataStore(token)
datastore.collections

In [ ]:
selected_collection = datastore.get_collection('EO:EUM:DAT:MSG:HRSEVIRI')
selected_collection?

##### Define download parameters & compile list of products

In [ ]:
# Add vertices for polygon, wrapping back to the start point.
geometry = [[-1.0, -1.0],[4.0, -4.0],[8.0, -2.0],[9.0, 2.0],[6.0, 4.0],[1.0, 5.0],[-1.0, -1.0]]

# Set sensing start and end time
start = datetime.datetime(2021, 10, 10, 9, 0)
end = datetime.datetime(2021, 10, 10, 10, 0)

# Retrieve datasets that match our filter
products = selected_collection.search(
    geo='POLYGON(({}))'.format(','.join(["{} {}".format(*coord) for coord in geometry])),
    dtstart=start, 
    dtend=end)
    
print(f'Found Datasets: {len(products)} datasets for the given time range')

for product in products:
    print(str(product))

##### Download products

Each product contains three files (EOPMetadata.xml, manifest.xml, and data.nat files). When downloading the whole product, a .zip folder is created.

In [ ]:
for product in products:
    with product.open() as fsrc, \
            open(fsrc.name, mode='wb') as fdst:
        shutil.copyfileobj(fsrc, fdst)
        print(f'Download of product {product} finished.')
print('All downloads are finished.')

In [24]:
for product in products:    
    print(product.url)

https://api.eumetsat.int/data/download/1.0.0/collections/EO%3AEUM%3ADAT%3AMSG%3AHRSEVIRI/products/MSG4-SEVI-MSG15-0100-NA-20211010095743.941000000Z-NA?access_token=b86efc5f-3691-3821-99c6-f7f8a6dc98d5
https://api.eumetsat.int/data/download/1.0.0/collections/EO%3AEUM%3ADAT%3AMSG%3AHRSEVIRI/products/MSG4-SEVI-MSG15-0100-NA-20211010094243.725000000Z-NA?access_token=b86efc5f-3691-3821-99c6-f7f8a6dc98d5
https://api.eumetsat.int/data/download/1.0.0/collections/EO%3AEUM%3ADAT%3AMSG%3AHRSEVIRI/products/MSG4-SEVI-MSG15-0100-NA-20211010092743.511000000Z-NA?access_token=b86efc5f-3691-3821-99c6-f7f8a6dc98d5
https://api.eumetsat.int/data/download/1.0.0/collections/EO%3AEUM%3ADAT%3AMSG%3AHRSEVIRI/products/MSG4-SEVI-MSG15-0100-NA-20211010091243.299000000Z-NA?access_token=b86efc5f-3691-3821-99c6-f7f8a6dc98d5


##### Download only selected products & files

In [25]:
selected_product = datastore.get_product(
    product_id='MSG4-SEVI-MSG15-0100-NA-20211110081242.766000000Z-NA',
    collection_id='EO:EUM:DAT:MSG:HRSEVIRI')

The data is stored in a .nat file

In [26]:
for entry in selected_product.entries:
    print(entry)

EOPMetadata.xml
MSG4-SEVI-MSG15-0100-NA-20211110081242.766000000Z-NA.nat
manifest.xml


In [11]:
try:
    with selected_product.open(entry='MSG4-SEVI-MSG15-0100-NA-20211110081242.766000000Z-NA.nat') as fsrc, \
            open(fsrc.name, mode='wb') as fdst:
        shutil.copyfileobj(fsrc, fdst)
        print(f'Download of file {fsrc.name} finished.')
except eumdac.product.ProductError as error:
    print(f"Error related to the product '{selected_product}' while trying to download it: '{error.msg}'")

Download of file MSG4-SEVI-MSG15-0100-NA-20211110081242.766000000Z-NA.nat finished.


In [27]:
try:
    with selected_product.open(entry='EOPMetadata.xml') as fsrc, \
            open(fsrc.name, mode='wb') as fdst:
        shutil.copyfileobj(fsrc, fdst)
        print(f'Download of file {fsrc.name} finished.')
except eumdac.product.ProductError as error:
    print(f"Error related to the product '{selected_product}' while trying to download it: '{error.msg}'")

Download of file EOPMetadata.xml finished.


Unfortunately, .nat files are a bit complicated to read/load. One of the best packages is satpy, which then allows loading the files using xarray.

In [4]:
from satpy import Scene
scn = Scene(reader="seviri_l1b_native", filenames=['MSG4-SEVI-MSG15-0100-NA-20211110081242.766000000Z-NA.nat'])

In [131]:
# MSG data contains 12 channels, including the high-resolution visible (HRV) channel.
datasets = scn.available_dataset_names()
datasets

['HRV',
 'IR_016',
 'IR_039',
 'IR_087',
 'IR_097',
 'IR_108',
 'IR_120',
 'IR_134',
 'VIS006',
 'VIS008',
 'WV_062',
 'WV_073']

In order to convert the scn object to xarray, each dataset needs to be loaded.

In [132]:
scn.load(datasets[1:], generate=False)
# Note: The HRV channel comes in 1 km resolution, while the other channels are 3 km resolution.
# Because of the size mismatch, the HRV channel would either need to be downscaled,
# or it needs to the loaded separetely to the other channels


In [133]:
ds = scn.to_xarray()

/opt/anaconda3/envs/iti-env/lib/python3.11/site-packages/satpy/cf/coords.py:201: UserWarning: Cannot pretty-format "acq_time" coordinates because they are not identical among the given datasets
  _warn_if_pretty_but_not_unique(pretty, coord_name)
